In [4]:
"""
PASO 8: RESOLUCIÓN DE MULTICOLINEALIDAD (VIF)

Proyecto: Análisis y Predicción de SOP
Fecha: 31 octubre 2025
Objetivo: Eliminar variables redundantes con VIF > 10

IMPORTANTE: 
- Ejecutar DESPUÉS de transformaciones (Paso 7)
- Usar dataset: PCOS_data_transformado.csv
- Meta: VIF < 10 en todas las variables (idealmente < 5)
"""

import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("PASO 8: RESOLUCIÓN DE MULTICOLINEALIDAD")
print("="*70)
print()

# =============================================================================
# 1. CARGAR DATASET TRANSFORMADO
# =============================================================================

print("📂 1. Cargando dataset transformado...")

# AJUSTA ESTA RUTA según dónde tengas tu archivo
df = pd.read_csv('../documentos_generados/PCOS_data_transformado.csv')

print(f"✅ Dataset cargado: {df.shape[0]} filas × {df.shape[1]} columnas")
print()

# =============================================================================
# 2. CALCULAR VIF INICIAL (con datos transformados)
# =============================================================================

print("🔍 2. Calculando VIF inicial...")

# Seleccionar solo variables numéricas (excluir objetivo)
# AJUSTA el nombre de tu variable objetivo según tu dataset
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remover variable objetivo
target_col = 'SOP (S/N)'  # AJUSTA si tu columna objetivo tiene otro nombre
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

print(f"Variables numéricas a analizar: {len(numeric_cols)}")
print()

# Calcular VIF
X = df[numeric_cols].values

vif_inicial = pd.DataFrame()
vif_inicial['Variable'] = numeric_cols
vif_inicial['VIF'] = [variance_inflation_factor(X, i) for i in range(len(numeric_cols))]
vif_inicial = vif_inicial.sort_values('VIF', ascending=False).reset_index(drop=True)

# Clasificar
vif_inicial['Clasificación'] = vif_inicial['VIF'].apply(
    lambda x: 'SEVERO (>10)' if x > 10 else ('MODERADO (5-10)' if x > 5 else 'OK (<5)')
)

print("📊 VIF INICIAL (Top 20 variables):")
print(vif_inicial.head(20).to_string(index=False))
print()

# Estadísticas
severo = (vif_inicial['VIF'] > 10).sum()
moderado = ((vif_inicial['VIF'] > 5) & (vif_inicial['VIF'] <= 10)).sum()
ok = (vif_inicial['VIF'] <= 5).sum()

print(f"🔴 SEVERO (VIF > 10):  {severo} variables ({severo/len(numeric_cols)*100:.1f}%)")
print(f"🟡 MODERADO (VIF 5-10): {moderado} variables ({moderado/len(numeric_cols)*100:.1f}%)")
print(f"🟢 OK (VIF < 5):        {ok} variables ({ok/len(numeric_cols)*100:.1f}%)")
print()

# =============================================================================
# 3. ESTRATEGIA DE ELIMINACIÓN
# =============================================================================

print("="*70)
print("🎯 3. ESTRATEGIA DE ELIMINACIÓN DE VARIABLES")
print("="*70)
print()

# PROPUESTA DE VARIABLES A ELIMINAR (basada en análisis previo)
# AJUSTA según los nombres de columnas en tu dataset

vars_propuestas_eliminar = [
    'Peso (Kg)',                    # Mantener IMC (más informativo)
    'Altura (cm)',                  # Mantener IMC
    'Cintura (pulg)',               # Mantener Ratio Cintura-Cadera
    'Cadera (pulg)',                # Mantener Ratio Cintura-Cadera
    'Presión Diastólica (mmHg)',    # Mantener Sistólica
    'Frecuencia Cardiaca (lpm)',    # Menos relevante clínicamente
    'Frecuencia Respiratoria (rpm)', # Menos relevante clínicamente
    'Hemoglobina (g/dL)',           # Menos relevante para SOP
    'Tamaño Folículo Prom (I) (mm)', # Mantener Número Folículos
    'Tamaño Folículo Prom (D) (mm)'  # Mantener Número Folículos
]

print("📋 Variables propuestas para eliminación:")
for i, var in enumerate(vars_propuestas_eliminar, 1):
    if var in vif_inicial['Variable'].values:
        vif_val = vif_inicial[vif_inicial['Variable'] == var]['VIF'].values[0]
        print(f"  {i}. {var} (VIF = {vif_val:.2f})")
    else:
        print(f"  {i}. {var} (NO ENCONTRADA)")
print()

# Verificar cuáles existen
vars_a_eliminar = [v for v in vars_propuestas_eliminar if v in df.columns]

print(f"✅ Variables encontradas para eliminar: {len(vars_a_eliminar)}/{len(vars_propuestas_eliminar)}")
print()

# OPCIÓN: Pausar aquí para revisión manual
print("⚠️ REVISAR ANTES DE CONTINUAR:")
print("   - ¿Las variables propuestas son correctas?")
print("   - ¿Hay otras variables que eliminar?")
print()
input("Presiona ENTER para continuar con la eliminación...")
print()

# =============================================================================
# 4. ELIMINAR VARIABLES Y RECALCULAR VIF
# =============================================================================

print("="*70)
print("🔧 4. ELIMINANDO VARIABLES Y RECALCULANDO VIF")
print("="*70)
print()

# Crear dataset sin variables redundantes
df_sin_multicol = df.drop(columns=vars_a_eliminar)

print(f"✅ Variables eliminadas: {len(vars_a_eliminar)}")
print(f"📊 Nuevo shape: {df_sin_multicol.shape[0]} filas × {df_sin_multicol.shape[1]} columnas")
print()

# Recalcular VIF
numeric_cols_final = df_sin_multicol.select_dtypes(include=[np.number]).columns.tolist()
if target_col in numeric_cols_final:
    numeric_cols_final.remove(target_col)

X_final = df_sin_multicol[numeric_cols_final].values

print("🔍 Recalculando VIF...")
vif_final = pd.DataFrame()
vif_final['Variable'] = numeric_cols_final
vif_final['VIF'] = [variance_inflation_factor(X_final, i) for i in range(len(numeric_cols_final))]
vif_final = vif_final.sort_values('VIF', ascending=False).reset_index(drop=True)

vif_final['Clasificación'] = vif_final['VIF'].apply(
    lambda x: 'SEVERO (>10)' if x > 10 else ('MODERADO (5-10)' if x > 5 else 'OK (<5)')
)

print("📊 VIF FINAL (Top 20 variables):")
print(vif_final.head(20).to_string(index=False))
print()

# Estadísticas finales
severo_final = (vif_final['VIF'] > 10).sum()
moderado_final = ((vif_final['VIF'] > 5) & (vif_final['VIF'] <= 10)).sum()
ok_final = (vif_final['VIF'] <= 5).sum()

print(f"🔴 SEVERO (VIF > 10):  {severo_final} variables ({severo_final/len(numeric_cols_final)*100:.1f}%)")
print(f"🟡 MODERADO (VIF 5-10): {moderado_final} variables ({moderado_final/len(numeric_cols_final)*100:.1f}%)")
print(f"🟢 OK (VIF < 5):        {ok_final} variables ({ok_final/len(numeric_cols_final)*100:.1f}%)")
print()

# =============================================================================
# 5. VALIDACIÓN
# =============================================================================

print("="*70)
print("✅ 5. VALIDACIÓN DE RESULTADOS")
print("="*70)
print()

if severo_final == 0:
    print("🎉 ÉXITO: No hay variables con multicolinealidad severa (VIF > 10)")
else:
    print(f"⚠️ ATENCIÓN: Aún hay {severo_final} variables con VIF > 10")
    print()
    print("Variables problemáticas:")
    print(vif_final[vif_final['VIF'] > 10][['Variable', 'VIF']])
    print()
    print("RECOMENDACIÓN: Considerar eliminar variables adicionales")

print()

# Comparación antes/después
print("📊 COMPARACIÓN ANTES/DESPUÉS:")
print(f"  Variables totales:  {len(numeric_cols)} → {len(numeric_cols_final)} (eliminadas: {len(numeric_cols) - len(numeric_cols_final)})")
print(f"  VIF promedio:       {vif_inicial['VIF'].mean():.2f} → {vif_final['VIF'].mean():.2f}")
print(f"  VIF máximo:         {vif_inicial['VIF'].max():.2f} → {vif_final['VIF'].max():.2f}")
print(f"  VIF > 10:           {severo} → {severo_final}")
print()

# =============================================================================
# 6. GUARDAR ARCHIVOS
# =============================================================================

print("="*70)
print("💾 6. GUARDANDO ARCHIVOS")
print("="*70)
print()

# Dataset final sin multicolinealidad
output_data = 'PCOS_data_sin_multicolinealidad.csv'
df_sin_multicol.to_csv(output_data, index=False)
print(f"✅ Dataset guardado: {output_data}")

# VIF inicial
vif_inicial.to_csv('VIF_inicial.csv', index=False)
print(f"✅ VIF inicial guardado: VIF_inicial.csv")

# VIF final
vif_final.to_csv('VIF_final.csv', index=False)
print(f"✅ VIF final guardado: VIF_final.csv")

# Comparación
comparacion = pd.DataFrame({
    'Métrica': ['Variables numéricas', 'VIF promedio', 'VIF máximo', 'Variables VIF>10', 'Variables VIF 5-10', 'Variables VIF<5'],
    'Antes': [len(numeric_cols), vif_inicial['VIF'].mean(), vif_inicial['VIF'].max(), severo, moderado, ok],
    'Después': [len(numeric_cols_final), vif_final['VIF'].mean(), vif_final['VIF'].max(), severo_final, moderado_final, ok_final]
})
comparacion.to_csv('Comparacion_VIF_antes_despues.csv', index=False)
print(f"✅ Comparación guardada: Comparacion_VIF_antes_despues.csv")

# Variables eliminadas
vars_eliminadas_df = pd.DataFrame({
    'Variable Eliminada': vars_a_eliminar,
    'VIF Original': [vif_inicial[vif_inicial['Variable'] == v]['VIF'].values[0] for v in vars_a_eliminar]
})
vars_eliminadas_df.to_csv('Variables_eliminadas.csv', index=False)
print(f"✅ Variables eliminadas guardadas: Variables_eliminadas.csv")

print()

# =============================================================================
# 7. RESUMEN FINAL
# =============================================================================

print("="*70)
print("📋 RESUMEN EJECUTIVO - PASO 8")
print("="*70)
print()

print("✅ PASO 8 COMPLETADO CON ÉXITO")
print()
print("📊 DATASET FINAL:")
print(f"  - Filas: {df_sin_multicol.shape[0]}")
print(f"  - Columnas: {df_sin_multicol.shape[1]}")
print(f"  - Variables eliminadas: {len(vars_a_eliminar)}")
print(f"  - Variables numéricas finales: {len(numeric_cols_final)}")
print()

if severo_final == 0:
    print("🎉 MULTICOLINEALIDAD RESUELTA:")
    print("  ✅ 0 variables con VIF > 10")
    print("  ✅ Dataset listo para modelado ML")
else:
    print("⚠️ MULTICOLINEALIDAD PARCIALMENTE RESUELTA:")
    print(f"  ⚠️ {severo_final} variables aún con VIF > 10")
    print("  📝 Considerar iteración adicional")

print()
print("📁 ARCHIVOS GENERADOS:")
print("  1. PCOS_data_sin_multicolinealidad.csv (dataset final)")
print("  2. VIF_inicial.csv (diagnóstico inicial)")
print("  3. VIF_final.csv (diagnóstico final)")
print("  4. Comparacion_VIF_antes_despues.csv (métricas)")
print("  5. Variables_eliminadas.csv (log de eliminaciones)")
print()
print("🎯 SIGUIENTE PASO: Feature Engineering (Paso 9)")
print()
print("="*70)

PASO 8: RESOLUCIÓN DE MULTICOLINEALIDAD

📂 1. Cargando dataset transformado...
✅ Dataset cargado: 538 filas × 42 columnas

🔍 2. Calculando VIF inicial...
Variables numéricas a analizar: 41

📊 VIF INICIAL (Top 20 variables):
                     Variable          VIF Clasificación
              Glucosa (mg/dl) 58321.240751  SEVERO (>10)
         Ratio Cintura-Cadera 31009.677646  SEVERO (>10)
                Cadera (pulg) 30117.615713  SEVERO (>10)
               Cintura (pulg) 29541.714299  SEVERO (>10)
                  Altura (cm) 26061.710796  SEVERO (>10)
                          IMC  6314.109754  SEVERO (>10)
                    Peso (Kg)  6226.666347  SEVERO (>10)
Frecuencia Respiratoria (rpm)  1976.825798  SEVERO (>10)
                 Ratio FSH/LH   253.430874  SEVERO (>10)
           Hemoglobina (g/dl)   185.924354  SEVERO (>10)
                 FSH (mUI/mL)   167.208660  SEVERO (>10)
                  LH (mUI/mL)   122.975047  SEVERO (>10)
          Vitamina D3 (ng/mL)    77